In [1]:
from pprint import pprint
import pymongo
from pymongo import MongoClient
import csv
from datetime import datetime
from datetime import timedelta
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import time

def samedate(hotspot_date, climate_date):
    # this function checks if two dates are the same
    # This function was made because date is forrmatted differently in hotspot_historic and climate_historic
    # eg. hotspot historic '27/12/2018' and climate historic '27/12/18'
    actual_date = []
    final_date = []
    join_string = ''
    date_length = len(hotspot_date)
    for k in range(date_length - 1, -1, -1):
        if k == 7 or k == 6:
            continue
        else:
            actual_date.append(hotspot_date[k])
    for j in range(date_length - 3, -1, -1):
        final_date.append(actual_date[j])

    return (join_string.join(final_date) == climate_date)

def get_prec_number(a_string):
    # this function gives the float part of a precipitation measurement
    final = a_string.strip()
    return float(final[0:4])
    

def get_prec_letter(a_string):
    # this function gives the letter representing the tyoe of precipitation measurement
    final = a_string.strip()
    return final[-1]

def change_format(a_date):
    date_final = datetime.strptime(a_date, '%d/%m/%y')
    formatted = date_final.strftime("20%y/%m/%d")
    return formatted

# Task 2
# Question 1
# So we have to load all the data from the csv files into a new database fit3182_assignment_db
# Check if the main collection already exists
try:
    a_collection.drop()
except NameError:
    pass
# Now create the data base
client = MongoClient() # so we connect on the default host and port
db = client.fit3182_assignment_db # create a new database fit3182_assignment_db
a_collection = db.climatee # create a new collection called 'a_collection'
with open('climate_historic.csv', 'r') as file_one:  # open climate historic file
    # now create a csv object for each of the files
    climate_data = csv.reader(file_one)
    climate_lines = []
    count = 0
    for c_line in climate_data:
        if count == 0:
            count += 1
        else:
            climate_lines.append(c_line)   # put each line of the data in a list 
            

            
with open('hotspot_historic.csv', 'r') as file_two: # open hotspot historic file
    hotspot_data = csv.reader(file_two)
    hotspot_lines = []
    count = 0
    for h_line in hotspot_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            hotspot_lines.append(h_line)  # put each line of the data in a list

documents = []
for climate_line in climate_lines:
    matched = False
    for hotspot_line in hotspot_lines:
        if samedate(hotspot_line[4], climate_line[1]):
            new_entry = {
             "latitude" : float(hotspot_line[0]),
             "longitude": float(hotspot_line[1]),
             "datetime" : hotspot_line[2],
             "confidence" : float(hotspot_line[3]),
             "surface_temperature_celsius" : float(hotspot_line[5]),
            "station": float(climate_line[0]),
            "date": change_format(climate_line[1]),
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])}
            documents.append(new_entry)
            matched = True
    if matched == False: # there was not a fire on this date
        # Set None's for all the unmatchable values cause there was no fire on this date
        no_fire = {
            "latitude" : None,
            "longitude": None,
            "datetime" : None,
            "confidence" : None,
            "surface_temperature_celsius": None,
            "station": float(climate_line[0]),
            "date": change_format(climate_line[1]),
            "air_temperature_celsius" : float(climate_line[2]),
            "relative_humidity": float(climate_line[3]),
            "windspeed_knots" : float(climate_line[4]),
            "max_wind_speed": float(climate_line[5]),
            "precipitation_level": get_prec_number(climate_line[6]),
            "precipitation_type": get_prec_letter(climate_line[6]),
            "GHI_w/m2": float(climate_line[7])
        }
        documents.append(no_fire)


result = a_collection.insert_many(documents)  
# cursor = a_collection.find({})
# for item in cursor:
#     pprint(item)
# find the latest date in the stream

latest_date = a_collection.find().sort("date", pymongo.DESCENDING).limit(1)
for k in latest_date:
    new_date = datetime.strptime(k['date'], '%Y/%m/%d')

In [ ]:
with open('hotspot_TERRA_streaming.csv', 'r') as file_four: # open hotspot historic file
    terra_data = csv.reader(file_four)
    terra_lines = []
    count = 0
    for line in terra_data:  # go through the hotspot data.
        if count == 0:
            count += 1
        else:
            terra_lines.append(line)  # put each line of the data in a list

def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ', data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'same_topic'
    prod = 'Producer3'
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    length_lines = len(terra_lines)
    start_time = time.time()
    while True:
        if time.time() - start_time <= 10:
            index_of_line = random.randrange(0, length_lines)
            new_date = new_date
            str_date = new_date.strftime('%Y/%m/%d')
            line_sent = terra_lines[index_of_line] + [str_date] + [prod]
            data = str(line_sent)
            publish_message(producer, topic, 'parsed', data)
            sleep(2)
        else:
            index_of_line = random.randrange(0, length_lines)
            new_date = new_date + timedelta(days = 1)
            str_date = new_date.strftime('%Y/%m/%d')
            line_sent = terra_lines[index_of_line] + [str_date] + [prod]
            data = str(line_sent)
            publish_message(producer, topic, 'parsed', data)
            start_time = time.time()
            sleep(2)
            

Publishing records..
Message published successfully. Data:  ['-36.7742', '145.1991', '100', '89', '2018/12/31', 'Producer3']
Message published successfully. Data:  ['-37.5673', '143.0535', '69', '44', '2018/12/31', 'Producer3']
Message published successfully. Data:  ['-36.1672', '145.8389', '81', '54', '2018/12/31', 'Producer3']
Message published successfully. Data:  ['-36.3045', '141.0771', '74', '48', '2018/12/31', 'Producer3']
Message published successfully. Data:  ['-36.1988', '145.0941', '100', '100', '2018/12/31', 'Producer3']
Message published successfully. Data:  ['-36.3126', '145.8104', '76', '63', '2019/01/01', 'Producer3']
Message published successfully. Data:  ['-36.4422', '141.427', '54', '39', '2019/01/01', 'Producer3']
Message published successfully. Data:  ['-37.856', '143.416', '75', '41', '2019/01/01', 'Producer3']
Message published successfully. Data:  ['-36.8847', '141.0956', '87', '62', '2019/01/01', 'Producer3']
Message published successfully. Data:  ['-36.4936', 

Message published successfully. Data:  ['-37.7741', '142.8188', '77', '50', '2019/01/16', 'Producer3']
Message published successfully. Data:  ['-37.4015', '145.5947', '88', '42', '2019/01/16', 'Producer3']
Message published successfully. Data:  ['-38.5214', '146.4027', '84', '57', '2019/01/16', 'Producer3']
Message published successfully. Data:  ['-37.6745', '142.9848', '75', '48', '2019/01/16', 'Producer3']
Message published successfully. Data:  ['-37.8277', '142.6216', '79', '52', '2019/01/16', 'Producer3']
Message published successfully. Data:  ['-37.5115', '143.14', '99', '86', '2019/01/17', 'Producer3']
Message published successfully. Data:  ['-37.926', '144.0898', '85', '59', '2019/01/17', 'Producer3']
Message published successfully. Data:  ['-36.6306', '143.8908', '68', '44', '2019/01/17', 'Producer3']
Message published successfully. Data:  ['-35.7626', '142.1167', '87', '62', '2019/01/17', 'Producer3']
Message published successfully. Data:  ['-36.1029', '145.0695', '66', '43', 

Message published successfully. Data:  ['-35.8125', '142.1286', '75', '109', '2019/02/01', 'Producer3']
Message published successfully. Data:  ['-36.3735', '147.3154', '86', '61', '2019/02/01', 'Producer3']
Message published successfully. Data:  ['-36.3851', '141.8256', '82', '55', '2019/02/01', 'Producer3']
Message published successfully. Data:  ['-36.1831', '142.0764', '80', '53', '2019/02/01', 'Producer3']
Message published successfully. Data:  ['-36.3438', '141.5832', '96', '78', '2019/02/01', 'Producer3']
Message published successfully. Data:  ['-36.7159', '141.4716', '51', '38', '2019/02/02', 'Producer3']
Message published successfully. Data:  ['-36.6564', '142.8615', '91', '69', '2019/02/02', 'Producer3']
Message published successfully. Data:  ['-36.4952', '143.971', '100', '110', '2019/02/02', 'Producer3']
Message published successfully. Data:  ['-36.6363', '144.3033', '54', '40', '2019/02/02', 'Producer3']
Message published successfully. Data:  ['-37.6622', '142.1001', '96', '

Message published successfully. Data:  ['-36.1574', '141.7206', '83', '57', '2019/02/17', 'Producer3']
Message published successfully. Data:  ['-36.6965', '142.7116', '88', '64', '2019/02/17', 'Producer3']
Message published successfully. Data:  ['-36.081', '145.5175', '85', '60', '2019/02/17', 'Producer3']
Message published successfully. Data:  ['-37.8918', '143.364', '76', '50', '2019/02/17', 'Producer3']
Message published successfully. Data:  ['-35.1175', '141.1396', '84', '58', '2019/02/17', 'Producer3']
Message published successfully. Data:  ['-36.6704', '141.3197', '69', '45', '2019/02/18', 'Producer3']
Message published successfully. Data:  ['-38.1403', '144.2187', '84', '58', '2019/02/18', 'Producer3']
Message published successfully. Data:  ['-36.533', '144.0183', '78', '51', '2019/02/18', 'Producer3']
Message published successfully. Data:  ['-36.3557', '141.3268', '62', '52', '2019/02/18', 'Producer3']
Message published successfully. Data:  ['-37.658', '149.339', '78', '36', '2